In [7]:
import jax
import torch
import pytensor
import pytensor.tensor as pt
import numpy as np

from pytensor.graph.fg import FunctionGraph
from pytensor.link.jax.dispatch import jax_funcify
from pytensor.link.pytorch.dispatch import pytorch_funcify
from pytensor.compile.mode import get_mode

from pytensor.graph.rewriting.utils import rewrite_graph

x = pt.vector("x")
one_mx = 1 - x
out = pt.log(one_mx)

fg = FunctionGraph(inputs=None, outputs=[out])

pytensor.dprint(fg)

Log [id A] 2
 └─ Sub [id B] 1
    ├─ ExpandDims{axis=0} [id C] 0
    │  └─ 1 [id D]
    └─ x [id E]


In [8]:
opt_fg = rewrite_graph(fg, include=("canonicalize", "stabilize", "specialize"))
pytensor.dprint(opt_fg)

Log1p [id A] 1
 └─ Neg [id B] 0
    └─ x [id C]


In [9]:
pytorch_fn = pytorch_funcify(opt_fg)
jax_fn = jax_funcify(opt_fg)

In [10]:
pytorch_compiled_fn = torch.compile(pytorch_fn)
pytorch_out = pytorch_compiled_fn(torch.tensor([0.9, 0.9]).cuda())[0]

jax_compiled_fn = jax.jit(jax_fn)
jax_out = jax_compiled_fn(np.array([0.9, 0.9]))[0]

print(f'JAX output = {jax_out}')
print(f'Pytorch output = {pytorch_out}')

JAX output = [-2.30258509 -2.30258509]
Pytorch output = tensor([-2.3026, -2.3026], device='cuda:0')


/home/harshvir/miniconda3/envs/pytensor-dev/lib/python3.11/site-packages/torch/_dynamo/utils.py:1764: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return node.target(*args, **kwargs)
/home/harshvir/miniconda3/envs/pytensor-dev/lib/python3.11/site-packages/torch/fx/interpreter.py:274: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return target(*args, **kwargs)
/home/harshvir/miniconda3/envs/pytensor-dev/lib/python3.11/site-packages/torch/fx/interpreter.py:274: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return target(*args, **kwargs)


In [11]:
??pytorch_fn

Signature: pytorch_fn(x)
Docstring: <no docstring>
Source:   
def pytorch_funcified_fgraph(x):
    # Neg(x)
    tensor_variable = elemwise_fn(x)
    # Log1p(Neg.0)
    tensor_variable_1 = elemwise_fn_1(tensor_variable)
    return (tensor_variable_1,)
File:      /tmp/tmpa33e4b_h
Type:      function

In [12]:
??jax_fn

Signature: jax_fn(x)
Docstring: <no docstring>
Source:   
def jax_funcified_fgraph(x):
    # Neg(x)
    tensor_variable = elemwise_fn(x)
    # Log1p(Neg.0)
    tensor_variable_1 = elemwise_fn_1(tensor_variable)
    return (tensor_variable_1,)
File:      /tmp/tmpldlcl44u
Type:      function